In [ ]:
# The purpose of this program is to combine any rows that have the same barcode (SHEETBAR). While filtering by the categorical river depth (CALCZCD) removed many duplicate barcodes that consisted of samples at the same site with varying CALCZCD values, there are still several samples that have the same barcode.

# It is important that our data sets consists only of unqiue barcodes because it removes issues when interpolating the data and predicting our missing continuous variable values. For example, if two samples have the same barcode, then their latitude and longitude points are going to be equal. Say one of these two records has a missing TP value. When the interpolation algorithm tries to predict the missing TP value, it will search for the closest samples and create a new TP value as a linear combination of those closets points. Since we have two identcal lat and long points, the distance will be 0 and thus, by definition of our weights in the linear combination, we will be diving by 0.

In [1]:
# Import libraries
import pandas as pd
from geopy import distance
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
# The data at this point will have already been filtered by its QF code and by its CALCZCD
# The data set should consist of only surface level points 
data = pd.read_csv("../pool data/ltrm_water_data_lat_long.csv")


/Users/alainastockdill/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,12,14,17,19,21,23,25,32,48,51,52,55,57,59,61,63,65,67,70,71,72,73,74,75,76,77,126,129) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Filter out the continous variables that we want
data = data[['SHEETBAR','TN','TP','TEMP','DO','TURB','COND','VEL','SS','WDP','CHLcal','SECCHI', 'LONGITUDE', 'LATITUDE', 'DATE']]

In [4]:
# Determine which columns are duplicates and count how many duplicates there are
duplicates = data.groupby(['SHEETBAR']).size()

In [5]:
# Reset index of duplicates and rename the column to "count" 
duplicates = pd.DataFrame(duplicates, columns = ['count']).reset_index()
duplicates = duplicates[duplicates['count'] > 1]

In [6]:
# Create empty dataframe that will store the new collapsed data set
collapsed_data = pd.DataFrame()

In [7]:
# Since we only need to examine the data that has duplicate barcodes, we can set aside the unique barcodes
collapsed_data = data[-pd.Series(data["SHEETBAR"]).isin(duplicates["SHEETBAR"])]

# Store the duplicated data in its own data set called data_dups
data_dups = data[pd.Series(data["SHEETBAR"]).isin(duplicates["SHEETBAR"])]


In [8]:
# We know based on inspection that there may be negative TP values. We are going to remove these samples
data_dups = data_dups.drop(data_dups.index[data_dups['TP'] < 0])

In [90]:
# Find the average of each variable by barcode
# If there is only na values, na will be returned
# For any number of values per variable, the average will be found
temp = data_dups.groupby(['SHEETBAR'], as_index = False).mean()

In [93]:
# For reference, this method of adding the date was not working
temp['DATE'] = data_dups['DATE'][temp['SHEETBAR']]

KeyError: "None of [Int64Index([-4604347, -4604346, -4604345, -4604343, -4604341, -4604339,\n            -4604337, -4604336, -4604335, -4604334,\n            ...\n            46020726, 46020728, 46020731, 46022545, 46022547, 46022567,\n            46025292, 47000506, 47000663, 47000678],\n           dtype='int64', length=40784)] are in the [index]"

In [87]:
# Create a separate dataframe that stores the date of each barcode 
# Drops duplicates so that we will only have one date per barcode
date = pd.DataFrame(data_dups, columns = ['SHEETBAR','DATE']).drop_duplicates()

In [89]:
# Adds the date to the collapsed data by the sheetbar
temp = temp.merge(date, on = ['SHEETBAR'])

In [84]:
# Add the combined duplicate samples to the single row samples
collapsed_data.append(temp)

,SHEETBAR,TN,TP,TEMP,DO,TURB,COND,VEL,SS,WDP,CHLcal,SECCHI,LONGITUDE,LATITUDE,DATE
0,-4604348,7.31000,0.471000,23.0,6.900000,225.00,600.000000,NaN,337.500,7.00,NaN,11.0,-90.533462,39.939235,06/11/1993
7,-4604344,6.32000,0.290000,20.2,8.000000,64.00,699.000000,NaN,86.800,5.00,NaN,18.0,-90.068509,40.316217,06/08/1993
10,-4604342,6.19000,0.417000,18.5,7.900000,100.00,688.000000,NaN,147.300,15.30,NaN,8.0,-89.651818,40.617643,06/07/1993
13,-4604340,2.88000,0.122000,16.0,8.200000,18.00,584.000000,NaN,24.500,0.40,NaN,-9.0,-89.956183,40.325077,06/07/1993
16,-4604338,4.50000,0.326000,19.2,7.000000,90.00,689.000000,NaN,109.500,18.00,NaN,16.0,-89.651818,40.617643,05/25/1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88589,46022567,2.29400,0.433000,28.7,9.700000,59.00,578.500000,0.01,44.800,0.83,69.883580,21.0,-90.507745,39.989032,08/02/2017
88591,46025292,2.58800,0.153000,11.2,10.300000,27.00,603.000000,0.02,22.280,1.50,3.617150,37.0,-90.049078,40.330350,11/12/2020
88593,47000506,1.30425,0.173250,10.4,12.166667,8.25,353.666667,0.05,11.475,NaN,8.780450,45.0,-91.205630,43.370779,10/29/1999
88597,47000663,1.68300,0.091000,15.6,9.400000,12.00,216.000000,NaN,16.100,NaN,32.310780,NaN,-91.049424,43.000181,06/07/2001
